# Analisis Data Mobil Bekas di Arab Saudi

## Pemahaman Bisnis

### Konteks

Pasar mobil bekas di Arab Saudi adalah sektor penting dalam industri otomotif negara tersebut. Banyak pembeli dan penjual menghadapi kesulitan dalam menentukan harga yang wajar untuk kendaraan bekas karena berbagai faktor yang mempengaruhi nilai mobil. Hal ini menyebabkan inefisiensi dalam pasar, yang berpotensi mengakibatkan kerugian finansial bagi penjual dan ketidakpercayaan di antara pembeli.

### Pernyataan Masalah

Dealer mobil bekas dan penjual individu di Arab Saudi menghadapi tantangan dalam menentukan harga mobil bekas secara akurat. Hal ini mengakibatkan:
1. Potensi kehilangan pendapatan karena harga yang terlalu rendah
2. Waktu penjualan yang lebih lama karena harga yang terlalu tinggi
3. Berkurangnya kepercayaan dan kepuasan pelanggan

### Stakeholder

- Dealer mobil bekas
- Penjual mobil individu
- Calon pembeli mobil
- Platform pasar mobil online (misalnya, [syarah.com](https://syarah.com/]))

### Goal

1. Mengembangkan model machine learning untuk memprediksi harga mobil bekas dengan margin kesalahan kurang dari 10%
2. Mengidentifikasi faktor-faktor kunci yang mempengaruhi harga mobil bekas di Arab Saudi
3. Menyediakan alat bagi stakeholder untuk memperkirakan nilai pasar yang wajar untuk mobil bekas

### Pendekatan Analitis

Kita akan menggunakan model regresi untuk memprediksi harga mobil bekas berdasarkan berbagai fitur seperti merek, model, tahun, jarak tempuh, dan atribut lainnya. Proyek ini akan melibatkan:
1. Pembersihan data dan preprocessing
2. Analisis eksploratori data untuk memahami hubungan antar fitur
3. Rekayasa fitur untuk menciptakan input yang relevan bagi model
4. Pemilihan dan pelatihan model
5. Evaluasi dan optimisasi model

### Metrik Evaluasi

Metrik utama:
- Root Mean Squared Error (RMSE)
- Mean Absolute Percentage Error (MAPE)

Metrik sekunder:
- R-squared (R²)
- Mean Absolute Error (MAE)

Kita akan fokus pada MAPE sebagai metrik utama karena memberikan persentase kesalahan yang mudah diinterpretasikan dalam konteks harga mobil.

### Dampak yang Diharapkan

- Penentuan harga mobil bekas yang lebih akurat dan konsisten
- Waktu penjualan yang lebih singkat bagi penjual
- Peningkatan kepercayaan dan transparansi di pasar mobil bekas
- Potensi peningkatan efisiensi pasar dan volume transaksi

Dengan mengatasi masalah bisnis ini, kita bertujuan untuk menciptakan nilai bagi semua pemangku kepentingan di pasar mobil bekas Arab Saudi, yang mengarah pada pasar yang lebih efisien dan terpercaya.